In [29]:
import codecs
import os
class BIOES:
    def __init__(self):
        self.q_dic = {}
        self.content_str=''
        self.tag_dic = {"disease": "疾病",
           "drug": "药物",
           "surgery": "手术",
           "symptom": "症状",
           "prevention": "预防措施",
           "inspection_method": "观察方法",
           "diagnostic_technology": "诊断技术",
           "non-drug_therapy": "物理疗法",
           "nursing_technology": "护理技术",
           "cause_of_disease": "病因",
           "pathological_mechanism": "病理",
           "medical_indicator": "医疗指标"}
    def ann2chara_tag(self,r_ann_path):
        self.q_dic = {}
        with codecs.open(r_ann_path, "r", encoding="utf-8") as f:#读取标签文件
            line = f.readline()
            line = line.strip("\n\r")
            while line != "":
                line_arr = line.split()
                print(line_arr)
                try:
                    cls = self.tag_dic[line_arr[1]] #line_arr[1]为标签名称 通过tag_dic 查字典后转化为中文标签
                    start_index = int(line_arr[2])#标签起始位置
                    end_index = int(line_arr[3])#标签的结束位置
                    length = end_index - start_index#标签长度
                    if length==1:
                        self.q_dic[start_index] = ("S-%s" % cls)
                    else:
                        for r in range(length):#为标签内的字标号
                            if r == 0:#第一个字标B
                                self.q_dic[start_index] = ("B-%s" % cls) #在字典内添加字标签 eg: 21：B-药物
                            elif r==length-1:
                                self.q_dic[start_index+r] = ("E-%s" % cls)
                            else:#标签内其余的标I
                                self.q_dic[start_index + r] = ("I-%s" % cls)#在字典内添加字标签 eg: 22：I-药物
                except:
                    line = f.readline()
                    line = line.strip("\n\r")
                    continue#出现异常则跳过 继续循环
                line = f.readline()
                line = line.strip("\n\r")
    def readtxt(self,r_txt_path):
        print("开始读取文件:%s" % r_txt_path)
        with codecs.open(r_txt_path, "r", encoding="utf-8") as f:#读取文本文档
            self.content_str = f.read()        
    def toBIOES(self,w_path):
        print("开始写入文本%s" % w_path)#开始将标签写入文本
        with codecs.open(w_path, "w", encoding="utf-8") as w:
            for i, str in enumerate(self.content_str):
                if str is " " or str == "" or str == "\n":
                    print("===============")
                elif str == "\r":
                    w.write("\r")
                else:
                    if i in self.q_dic:#文本中的位置若存在标签 赋给tag标签
                        tag = self.q_dic[i]
                    else:
                        tag = "O"  # 大写字母O
                    w.write('%s %s\n' % (str, tag))#每个字一行每个字都有标签 分隔符在此设置
                    if str=="。"or str=="?"or str=="!":#输入要求 保证每一句话后都有一个空格
                        w.write("\n")
            w.write('%s\n' % "END O")
    def combine(self,data_root_dir, w_path):#合并文件 生成训练测试集
        if os.path.exists(w_path):
            os.remove(w_path)
        for file in os.listdir(data_root_dir):
            path = os.path.join(data_root_dir, file)
            if file.endswith("abstract_007-new.txt"):
                # 重命名为dev.txt
                os.rename(path, os.path.join(data_root_dir, "dev.txt"))
                continue
            #if file.endswith("abstract_011-new.txt"):
                # 重命名为test.txt
                #os.rename(path, os.path.join(data_root_dir, "test.txt"))
               # continue
            q_list = []
            print("开始读取文件:%s" % file)
            with codecs.open(path, "r", encoding="utf-8") as f:
                line = f.readline()
                line = line.strip("\n\r")
                while line != "END O":
                    q_list.append(line)
                    line = f.readline()
                    line = line.strip("\n\r")
            print("开始写入文本%s" % w_path)
            with codecs.open(w_path, "a", encoding="utf-8") as f:
                for item in q_list:
                    f.write('%s\n' % item)
    def run(self):
        data_dir = "datas"
        for file in os.listdir(data_dir):
            if file.find(".") == -1:
                continue
            file_name = file[0:file.find(".")]
            r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
            r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
            w_path = "%s/new/%s-new.txt" % (data_dir, file_name)
            self.ann2chara_tag(r_ann_path)
            self.readtxt(r_txt_path)
            self.toBIOES(w_path)
            # 生成train.txt、dev.txt、test.txt
        self.combine("%s/new" % data_dir, "%s/new/train.txt" % data_dir)